---
title: "Pandas: Transform like behavior using data from multiple columns"
author: "Damien Martin"
date: "2024-07-03"
categories: [pandas]
---

# The problem

We have a couple of different patterns for aggregating data collections:

- `groupby([group_cols]).apply`: Allows us to group, and then apply an arbitrary function on the group. Resulting dataframe will have a different index (the grouped variable).
- `groupby([group_cols])['column'].transform`: Allows us to group, and then apply a transformation on the `column` data, and return the aggregate result for each row. ie. the index returned will be the same (we preserve rows)

For example, if we 